In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline


from sklearn.model_selection import *
from sklearn.metrics import *
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
import os

from random import *

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# define train set
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('./drive/My Drive/DACON/data_file/train.csv')
test = pd.read_csv('./drive/My Drive/DACON/data_file/test.csv')
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
rot_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=45, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     # 블로그 펌 ㅎ
    zoom_range=0,      # 마찬가지 블로그 펌 ㅎ
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

trans_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.2, # 블로그 펌
    height_shift_range=0.2,# 블로그 펌
    brightness_range=None,
    shear_range=0,     # 블로그 펌
    zoom_range=0,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

shear_zoom_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.2,     # 블로그 펌
    zoom_range=0.2,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

flip_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     # 블로그 펌
    zoom_range=0,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=True, # 뒤집기
    vertical_flip=True,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

In [ ]:
def augmentation( input_imgs ):
    df = input_imgs
    new_data_set = []
    num_of_training_set = df.shape[0]

    for i in range(num_of_training_set//2):
        rand_1 = np.random.randint(num_of_training_set)
        rand_2 = np.random.randint(num_of_training_set)
        rand_3 = np.random.randint(num_of_training_set)
        rand_4 = np.random.randint(num_of_training_set)
    
        for j in range(3):
            # rotation
            _rot = rot_gen.flow( np.array(df.iloc[rand_1,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_1,1],
                df.iloc[rand_1,2],
            ] + list(_rot)]
            # translation
            _trans = trans_gen.flow( np.array(df.iloc[rand_2,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_2,1],
                df.iloc[rand_2,2],
            ] + list(_trans)]
            # shear / zoom
            _shear = shear_zoom_gen.flow( np.array(df.iloc[rand_3,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_3,1],
                df.iloc[rand_3,2],
            ] + list(_shear)]
            # flip
            _flip = flip_gen.flow( np.array(df.iloc[rand_4,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_4,1],
                df.iloc[rand_4,2],
            ] + list(_flip)]

    columns = ['digit', 'letter'] + [str(x) for x in range(784)]
    aug = pd.DataFrame(new_data_set, columns=columns)

    train_norm = pd.concat([ input_imgs.iloc[:,1:3], np.divide(input_imgs.iloc[:,3:],255) ],axis=1)
    train_aug = pd.concat([train_norm,aug])

    return train_aug


In [ ]:
def train_test_gen(input_imgs):
    train_aug = augmentation(input_imgs)

    x_train = train_aug.iloc[:,2:].values.copy()
    x_train = x_train.reshape(-1,28,28,1)

    y_train = train_aug['digit']
    y_train = to_categorical(y_train,num_classes = 10)

    return train_test_split(x_train,y_train,test_size=0.1,random_state=randint(1,100))

In [ ]:
def load_best(file_name):
    filepath = './drive/My Drive/DACON/saved_model/' + file_name + '/'
    time_list = []
    for f_name in os.listdir(f"{filepath}"):
        written_time = os.path.getctime(f"{filepath}{f_name}")
        time_list.append((f_name, written_time))
    sorted_file_list = sorted(time_list, key=lambda x: x[1], reverse=True)
    best = sorted_file_list[0]
    best_name = best[0]
    model = load_model( filepath + best_name )
    print('\033[31m' + best_name + '\033[0m')
    print()
    return model

In [ ]:
def get_model(N):
    
    model = Sequential()

    model.add(Conv2D(filters = N, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu', input_shape = (28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = N, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu'))
    model.add(BatchNormalization())
                
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 2*N, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 2*N, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(4*N, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = "softmax"))

    return model

In [ ]:
def ensemble(input_imgs, model_list):
    pred = []
    L = input_imgs.shape[0]
    label_list = np.zeros((L,10))
    num = len(model_list)
    for i in range(num):
        label = model_list[i].predict_on_batch( np.array(input_imgs).reshape(-1,28,28,1).astype(np.float32) )
        # label_list += label
        label_list += label*(1+i*0.2)
    for j in range(len(label_list)):
        pred.append( np.argmax(label_list[j]) )

    return pred

In [ ]:
def compare(file1,file2):
    filepath1 = './drive/My Drive/DACON/submission/' + file1 +'.csv'
    filepath2 = './drive/My Drive/DACON/submission/' + file2 +'.csv'
    f1 = pd.read_csv(filepath1)
    f2 = pd.read_csv(filepath2)
    match = np.array( [ f1['digit']==f2['digit'] ][0] )
    acc = len( np.where(match==True)[0] )/len(match)
    return acc

In [ ]:
def pred_acc(file_name,file_list):
    score = []
    for i in range( len(file_list) ):
        acc = compare(file_name, file_list[i])
        score.append(acc)
        print( 'Compared with ' + file_list[i].replace('submision_','') + ' : {}'.format(acc) )
    #return score
    

In [ ]:
def set_folder_path(folder_name):
    MODEL_SAVE_FOLDER_PATH = './drive/My Drive/DACON/saved_model/'+ folder_name +'/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)
    return MODEL_SAVE_FOLDER_PATH

# Re-run part

In [ ]:
epochs = 100
batch_size = 100
folder_name = 'model_boosting'
model_list = []
acc_list = []


iter = 10
w=2

for i in range(iter):

    print('\033[31m' + 'Boosting : {}'.format(i) +  '\033[0m')

    # define train/test data
    train_aug = augmentation(train_copy)
    x_train = train_aug.iloc[:,2:].values.copy()
    x_train = x_train.reshape(-1,28,28,1)
    y_train = train_aug['digit']
    y_train = to_categorical(y_train,num_classes = 10)

    if i>0 :
        x_error =  x_val[error_id]
        y_error = y_val[error_id]
        for i in range(w): 
            x_train = np.concatenate( (x_train,x_error),axis=0 )
        for i in range(w):
            y_train = np.concatenate( (y_train,y_error), axis=0 )

    x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=randint(1,100))

    # create model
    model = get_model(64)
    model_path = set_folder_path(folder_name) + 'model_{val_accuracy:.4f}.hdf5'

    # callbacks
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max')
    mcp_save = ModelCheckpoint(filepath = model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='min')
    # compile
    optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
    # fit model
    hist = model.fit(x_train, y_train, batch_size=batch_size, epochs = epochs, 
                validation_data = (x_val,y_val),
                steps_per_epoch=x_train.shape[0]// batch_size, 
                callbacks=[early_stopping,mcp_save,reduce_lr_loss])
    model = load_best(folder_name)
    model_list.append(model)
    acc_list.append(hist.history['val_accuracy'][-11])

    prob = model.predict_on_batch( x_val )
    pred = []
    val = []
    for i in range(len(prob)):
        pred.append( np.argmax(prob[i]) )
        val.append( np.argmax(y_val[i]) )
    error = [ np.array(pred)!=np.array(val) ]
    error_id = np.where(error[0]==True)[0]

    print('\033[31m' + 'val_accuracy : {}'.format(hist.history['val_accuracy'][-11]) +  '\033[0m')
    print()


In [ ]:
x_test = np.divide(test_copy.iloc[:,2:].values,255)
x_test = x_test.reshape(-1,28,28,1)
pred = ensemble(x_test, model_list)
data = {'id':test_copy['id'], 'digit':pred}
submission = DataFrame(data)

file_name = 'submission_cnn_boosting_10_w4'
submission.to_csv('./drive/My Drive/DACON/submission/'+ file_name +'.csv', index=False)
file_list = [ 'submission_84',
             'submission_85',
             'submission_86_xgb_ensemble',
             'submission_87_ensembles',
             'submission_88_ensemble_2_2_4_try3',
             'submission_89_ensemble_2_2',
             'submission_89',
             'submission_91_ensembles_3+1_w1',
             'submission_88_cnn_boosting_3_w2',
             'submission_cnn_boosting_3_w3'
             ]
pred_acc(file_name,file_list)

In [ ]:
compare(file_name,'submission_88_cnn_boosting_10_w4')